In [ ]:
import pandas as pd
import numpy as np 
import utm 
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import cartopy.io.img_tiles as cimgt
import cartopy
import cartopy.crs as ccrs
import cartopy.geodesic as cgeo
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pyproj
from matplotlib import pyplot as plt, patches
import matplotlib.patches as mpatches
import pickle
from matplotlib.patches import Polygon
from matplotlib.colors import Normalize
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from scipy.optimize import lsq_linear
import pickle
import sys
sys.path.insert(1, '/path/to/vsm/VSM')
import VSM_forward
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
warnings.filterwarnings("ignore", category=UserWarning, module="cartopy")
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import h5py

def read_filter_mmr(file, window_size, polynomial_order):
    from scipy.signal import savgol_filter
    import pandas as pd
    from pyproj import Proj
   
    outline = pd.read_csv(file, header=None)
    x = outline[1]
    y = outline[0]
      
    yhat = savgol_filter(y, window_size, polynomial_order) # window size 21, polynomial order 2
    xhat = savgol_filter(x, window_size, polynomial_order)
   
    x_norm = 419691.465854
    y_norm = 5081627.38769
   
    xhat_utm = [x + x_norm for x in xhat]
    yhat_utm = [y + y_norm for y in yhat]
      
    myProj = Proj("+proj=utm +zone=9 +north +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
    xhat_lon, yhat_lat = myProj(xhat_utm, yhat_utm, inverse=True)
   
    return xhat_lon, yhat_lat

def utm_to_latlon(x, y, zone_number, zone_letter):
    """
    Convert UTM coordinates to latitude and longitude.
    
    Parameters:
    - x, y: UTM coordinates
    - zone_number: UTM zone number
    - zone_letter: UTM zone letter

    Returns:
    - lat, lon: Latitude and longitude
    """

    utm_proj = f'+proj=utm +zone={zone_number} +ellps=WGS84 +units=m +no_defs'
    if zone_letter < 'N':
        utm_proj += ' +south'
    
    utm = pyproj.Proj(utm_proj)
    wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')
    
    transformer = pyproj.Transformer.from_proj(utm, wgs84)
    
    lon, lat = transformer.transform(x, y)

    return lat, lon

def utm_to_latlon_vec(x_array, y_array, zone_number, zone_letter):
    latitudes = []
    longitudes = []
    for x, y in zip(x_array, y_array):
        lat, lon = utm_to_latlon(x, y, zone_number, zone_letter)
        latitudes.append(lat)
        longitudes.append(lon)
    return longitudes, latitudes

def latlon_to_utm(lat, lon, zone_number, zone_letter):
    """
    Convert latitude and longitude to UTM coordinates.
    
    Parameters:
    - lat, lon: Latitude and longitude
    - zone_number: UTM zone number
    - zone_letter: UTM zone letter

    Returns:
    - x, y: UTM coordinates
    """

    utm_proj = f'+proj=utm +zone={zone_number} +ellps=WGS84 +units=m +no_defs'
    if zone_letter < 'N':
        utm_proj += ' +south'
    
    utm = pyproj.Proj(utm_proj)
    wgs84 = pyproj.Proj(proj='latlong', datum='WGS84')
    
    transformer = pyproj.Transformer.from_proj(wgs84, utm)
    
    x, y = transformer.transform(lon, lat)

    return x, y

def get_displacement_array(X, Y, patch, m):
    _, _, uz = VSM_forward.okada(X, Y, 
                     patch['xtlc'], patch['ytlc'], patch['dtlc']*-1, 
                     patch['length'], patch['width'], 
                     patch['strike'], patch['dip'], 
                     0.0, 0.0, m, 'R', 0.25)
    return uz

# Read in files

In [ ]:
mmr_x, mmr_y = read_filter_mmr('../data/mmr_2d_outline.csv', 21, 2)

bathy = pd.read_csv('../data/GMRTv3_9_20211012topo_cut2.xyz', header = None, \
                    delim_whitespace=True)
bathy.columns = ['lon', 'lat', 'z']
bathy = bathy.astype('float32')
bathy_z = bathy['z'].values.reshape(866,640)
bathy_x = bathy['lon'].values.reshape(866,640)
bathy_y = bathy['lat'].values.reshape(866,640)

# Load mmr point cloud for plotting
data = np.loadtxt('../data//axial_amc_top_utm_norm_km_20000.xyz')
xm = data[:, 0]
ym = data[:, 1]
x_norm = 419691.465854
y_norm = 5081627.38769
xm = [x + x_norm for x in xm]
ym = [y + y_norm for y in ym]
zm = data[:, 2]
zm = [(z - 20000+1500) for z in zm] # normalize z coordinate 

# load rectangles geometry, also just for plotting later. this was created by model3b_voxel_grid.ipynb
with open('modified_rectangles.pkl', 'rb') as f:
    rectangles = pickle.load(f)

# Read the patches CSV into a DataFrame. This is different from the rectangles because it constains 
# what the okada function needs (rectangle centers not corners). also created by model_3b_voxel_grid.ipynb
df = pd.read_csv('patches.csv')
# Convert to a list of dictionaries
patches = df.to_dict(orient='records')

# Read in deformation data
mpr_file = '../data/mpr_2016_2020_utm.txt'
auv_file = '../data/auv_2016_2020_utm_er.xyz'

column_names = ["x", "y", "z_displacement", "error"]

mpr = pd.read_csv(mpr_file, delim_whitespace=True, names=column_names)
auv = pd.read_csv(auv_file, delim_whitespace=True, names=column_names)
auv['error'] = auv['error'].replace(0.02, 0.01)


# Joint Inversion

In [ ]:
observations_df = pd.concat([mpr, auv], ignore_index=True)
observations = list(zip(observations_df["x"], observations_df["y"]))

In [ ]:
num_patches = len(df)
adjacency_matrix = np.zeros((num_patches, num_patches))
def are_adjacent(patch1, patch2):
    x_distance = abs(patch1['xtlc'] - patch2['xtlc'])
    y_distance = abs(patch1['ytlc'] - patch2['ytlc'])
    
    buffer = 1e-6
    
    if (x_distance < patch1['width'] + buffer and y_distance < buffer) or \
       (y_distance < patch1['length'] + buffer and x_distance < buffer):
        return True
    return False

for i in range(num_patches):
    for j in range(i+1, num_patches):  # we only need to check half the matrix
        if are_adjacent(df.iloc[i], df.iloc[j]):
            adjacency_matrix[i, j] = -1
            adjacency_matrix[j, i] = -1

for i in range(num_patches):
    adjacency_matrix[i, i] = -adjacency_matrix[i].sum()

# Construct the Green's function with weights

Start out by weighing the datasets evenly - both contribute equally to the inversion based on their uncertainties and number of data points. The MPR data is weighted heavier based on its uncertainty. 

\begin{align*}
        \text{Weight for each AUV point} &: \frac{1}{0.2} = 5 \\
        \text{Weight for each MPR point} &: \frac{1}{0.01} = 100
    \end{align*}


   \begin{align*}
        \text{Total AUV weight} &: 5 \times 1006 = 5030 \\
        \text{Total MPR weight} &: 100 \times 9 = 900
    \end{align*}
    

  \begin{align*}
        \text{Scaling factor for AUV} &= \frac{\text{Desired total weight for AUV}}{\text{Original total weight for AUV}} \\
        &= \frac{900}{5030} \approx 0.179
    \end{align*}

   \begin{align*}
        \text{Adjusted weight for each AUV point} &: 5 \times 0.179 = 0.895 \\
        \text{Weight for each MPR point} &: \text{remains unchanged at } 100
    \end{align*}



\begin{align*}
    \text{For the AUV data} &: 0.895 \text{ for each data point} \\
    \text{For the MPR data} &: 100 \text{ for each data point}
\end{align*}

In [ ]:
num_observations = len(observations)

# Create an empty Green's function matrix
G = np.zeros((num_observations, num_patches))

# Set weights
error_weights = 1 / (observations_df["error"])
mpr_error_weight = error_weights[:len(mpr)][0]
auv_error_weight = error_weights[len(mpr):len(mpr)+1].values[0]
total_mpr_weight = mpr_error_weight * len(mpr)
total_auv_weight = auv_error_weight * len(auv)
scaling_factor_auv = total_mpr_weight / total_auv_weight

error_weights[len(mpr):] *= scaling_factor_auv 

d = np.array(observations_df["z_displacement"]) * error_weights

for j, patch in enumerate(patches):
    x_obs_array, y_obs_array = zip(*observations)
    _, _, uz_array = VSM_forward.okada(np.array(x_obs_array), np.array(y_obs_array), 
                           patch['xtlc'], patch['ytlc'], patch['dtlc']*-1, 
                           patch['length'], patch['width'], 
                           patch['strike'], patch['dip'], 
                           0.0, 0.0, 1.0, 'R', 0.25)
    G[:, j] = uz_array * error_weights
    
error_weights

# Find optimal smoothing using L-curve

In [ ]:
lambda_values = np.logspace(-1, 1, 25)
d_original = np.array(observations_df["z_displacement"])

model_norms = []
weighted_rmses = []
converged_lambda_values = []
rmses_mpr =[]
rmses_auv = []

for lambda_smoothing in lambda_values:
    G_extended = np.vstack([G, lambda_smoothing * adjacency_matrix])
    d_extended = np.hstack([d, np.zeros(num_patches)])
    lower_bounds = np.zeros(num_patches)
    upper_bounds = np.inf * np.ones(num_patches)
    res = lsq_linear(G_extended, d_extended, bounds=(lower_bounds, upper_bounds), lsmr_tol='auto')
    m = res.x
    if res.success:
        converged_lambda_values.append(lambda_smoothing)

    else:
        print(f"Solution did not converge for lambda = {lambda_smoothing:.2e}. Message: {res.message}")
    d_syn = np.zeros(num_observations)
    x_obs_array, y_obs_array = zip(*observations)
    for j, patch in enumerate(patches):
        _, _, uz_array = VSM_forward.okada(np.array(x_obs_array), np.array(y_obs_array), 
                               patch['xtlc'], patch['ytlc'], patch['dtlc']*-1, 
                               patch['length'], patch['width'], 
                               patch['strike'], patch['dip'],
                               0.0, 0.0, m[j], 'R', 0.25)
        d_syn += uz_array
    
    model_norm = np.linalg.norm(m)
    model_norms.append(model_norm)
    residuals = d_original - d_syn
    residuals_mpr = residuals[:len(mpr)]
    residuals_auv = residuals[len(mpr):]

    rmse_mpr = np.sqrt(np.mean(residuals_mpr**2))
    rmse_auv = np.sqrt(np.mean(residuals_auv**2))
    rmses_mpr.append(rmse_mpr)
    rmses_auv.append(rmse_auv)
    
    weights_mpr = error_weights[:len(mpr)]
    weighted_rmse_mpr = np.sqrt(np.sum(weights_mpr * residuals_mpr**2) / np.sum(weights_mpr))
    
    weights_auv = error_weights[len(mpr):]
    weighted_rmse_auv = np.sqrt(np.sum(weights_auv * residuals_auv**2) / np.sum(weights_auv))
    
    normalized_weighted_avg_rmse = (weighted_rmse_mpr * len(mpr) + weighted_rmse_auv * len(observations_df[len(mpr):])) / len(observations_df)
    weighted_rmses.append(normalized_weighted_avg_rmse)

In [ ]:
corner_index = 10

optimal_mpr_rmse = rmses_mpr[corner_index]
optimal_auv_rmse = rmses_auv[corner_index]
optimal_norm = model_norms[corner_index]
optimal_lambda = lambda_values[corner_index]

from matplotlib.ticker import FuncFormatter

fig = plt.figure(figsize=(7,3))
ax1 = fig.add_subplot(1, 2, 1)
plt.plot(model_norms, rmses_auv, 'o-')
plt.xlabel('Model '+r'$L_2$ norm (roughness)')
plt.ylabel('RMSE (m)')
plt.title('a. AUV L-curve')

num_ticks = 6


ax1 = plt.gca()
ax1.xaxis.set_major_locator(MaxNLocator(num_ticks))

plt.xticks(rotation=45)
plt.grid(True, which="both", ls="--")

plt.scatter([optimal_norm], [optimal_auv_rmse], color='red', s=50, zorder=5)  


plt.annotate(f'Optimal smoothing \n($\lambda$): {optimal_lambda:.2f}', 
             (optimal_norm, optimal_auv_rmse),
             xytext=(1.05*optimal_norm, 1.07*optimal_auv_rmse), 
             arrowprops=dict(facecolor='black', arrowstyle='-|>', mutation_scale=30, shrinkB=7),
             fontsize=10);

ax2 = fig.add_subplot(1, 2, 2)
plt.plot(model_norms, rmses_mpr, 'o-')
plt.xlabel('Model '+r'$L_2$ norm (roughness)')
plt.ylabel('RMSE (m)')
plt.title('b. MPR L-curve')

num_ticks = 6

ax2 = plt.gca()
ax2.xaxis.set_major_locator(MaxNLocator(num_ticks))

plt.xticks(rotation=45)
plt.grid(True, which="both", ls="--")

plt.scatter([optimal_norm], [optimal_mpr_rmse], color='red', s=50, zorder=5)  
 
plt.annotate(f'Optimal smoothing \n($\lambda$): {optimal_lambda:.2f}', 
             (optimal_norm, optimal_mpr_rmse),
             xytext=(1.05*optimal_norm, 2*optimal_mpr_rmse), 
             arrowprops=dict(facecolor='black', arrowstyle='-|>', mutation_scale=30, shrinkB=7),
             fontsize=10);

fig.subplots_adjust(wspace=0.5)
plt.tight_layout()
plt.savefig('L_Curve_2016_2020.png', dpi=300)

# Do bounded least squares inversion using optimal smoothing 

In [ ]:
G = np.zeros((num_observations, num_patches))


d = np.array(observations_df["z_displacement"]) * error_weights

for j, patch in enumerate(patches):
    x_obs_array, y_obs_array = zip(*observations)
    _, _, uz_array = VSM_forward.okada(np.array(x_obs_array), np.array(y_obs_array), 
                           patch['xtlc'], patch['ytlc'], patch['dtlc']*-1, 
                           patch['length'], patch['width'], 
                           patch['strike'], patch['dip'], 
                           0.0, 0.0, 1.0, 'R', 0.25)
    G[:, j] = uz_array * error_weights

G_extended = np.vstack([G, optimal_lambda * adjacency_matrix])
d_extended = np.hstack([d, np.zeros(num_patches)])
lower_bounds = np.zeros(num_patches)
upper_bounds = np.inf * np.ones(num_patches)

res = lsq_linear(G_extended, d_extended, bounds=(lower_bounds, upper_bounds), lsmr_tol='auto')
m = res.x

dv = [patch['length'] * patch['width'] * slip for patch, slip in zip(patches, m)]
dv_tot =  sum(dv)/1e9
print("Volume change: {:.6f} km\u00B3".format(dv_tot))

# Generate gridded synthetic displacements and calculate rmse

In [ ]:
lon_min, lon_max, lat_min, lat_max = -130.0908385, -129.831339125, 45.8310127, 46.0402239975

x_min, y_min, _, _ = utm.from_latlon(lat_min, lon_min)
x_max, y_max, _, _ = utm.from_latlon(lat_max, lon_max)

resolution = 20
x = np.arange(x_min, x_max + resolution, resolution)
y = np.arange(y_min, y_max + resolution, resolution)
X, Y = np.meshgrid(x, y)

Z = np.zeros_like(X)
for j, patch in enumerate(patches):
    Z += get_displacement_array(X, Y, patch, m[j])
    
# Horizontal displacements
UX = np.zeros_like(X)
UY = np.zeros_like(X)
for j, patch in enumerate(patches):
    ux_flat, uy_flat, _ = VSM_forward.okada(
        X.ravel(), Y.ravel(),
        patch['xtlc'], patch['ytlc'], patch['dtlc'] * -1,
        patch['length'], patch['width'],
        patch['strike'], patch['dip'],
        0.0, 0.0, m[j], 'R', 0.25
    )
    UX += ux_flat.reshape(X.shape)
    UY += uy_flat.reshape(X.shape)

d_original = np.array(observations_df["z_displacement"])

d_syn = np.zeros(num_observations)
x_obs_array, y_obs_array = zip(*observations)

for j, patch in enumerate(patches):
    _, _, uz_array = VSM_forward.okada(np.array(x_obs_array), np.array(y_obs_array), 
                           patch['xtlc'], patch['ytlc'], patch['dtlc']*-1, 
                           patch['length'], patch['width'], 
                           patch['strike'], patch['dip'],
                           0.0, 0.0, m[j], 'R', 0.25)
    d_syn += uz_array
    

residuals = d_original - d_syn
combined_weighted_rmse = np.sqrt(np.sum(error_weights * residuals**2) / np.sum(error_weights))
print("Combined rmse: "+str(combined_weighted_rmse))

residuals_mpr = residuals[:len(mpr)]
residuals_auv = residuals[len(mpr):]

rmse_mpr = np.sqrt(np.mean(residuals_mpr**2))
rmse_auv = np.sqrt(np.mean(residuals_auv**2))

print("RMSE for mpr:", rmse_mpr)
print("RMSE for auv:", rmse_auv)

# Plot results

In [ ]:
norm = mcolors.Normalize(vmin=min(m), vmax=max(m))
colormap = plt.get_cmap('hot_r')

zone_number = 9  
zone_letter = 'T'  

lat, lon = utm_to_latlon(X, Y, zone_number, zone_letter)

vmn = -5
vmx = 5
colornum = 30
levels = np.linspace(-2500, 1300, 140) 
xlim = [-130.067, -129.91]
ylim = [45.87, 46.04]
cm = 'viridis'

fig = plt.figure(figsize=(20,8))
fig.suptitle("MMR 3D Joint Inversion", fontsize=24)


#------------------------------------------------------------------------------------------------


ax = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())

d = plt.contour(bathy_x, bathy_y, bathy_z, levels, transform = ccrs.PlateCarree(),colors='black', \
               linewidths = 1, linestyles='solid', zorder=2, alpha = .6);


ax = plt.gca()
ax.set_xlim(xlim)
ax.set_ylim(ylim);

ax.set_yticks(np.linspace(ylim[0]+0.02, ylim[1]-0.01, 4), crs = ccrs.PlateCarree())
lat_formatter = LatitudeFormatter()
ax.yaxis.set_major_formatter(lat_formatter)
ax.set_xticks(np.linspace(xlim[0]+0.02, xlim[1]-0.015,4), crs = ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
ax.xaxis.set_major_formatter(lon_formatter)
plt.yticks(rotation=90)


plt.plot(mmr_x, mmr_y,'--', transform = ccrs.PlateCarree(), color='black', alpha=1, linewidth = 2)

for idx, rectangle in enumerate(rectangles):
    latlon_rectangle = []
    
    for vertex in rectangle:
        x, y, _ = vertex
        latitude, longitude = utm_to_latlon(x, y, zone_number, zone_letter)
        latlon_rectangle.append((latitude, longitude))
    
    latitudes = [point[0] for point in latlon_rectangle]
    longitudes = [point[1] for point in latlon_rectangle]
    
    color = colormap(norm(m[idx]))
    polygon = Polygon(list(zip(longitudes, latitudes)), fill=True, facecolor=color, \
                      edgecolor='black', alpha=0.8, linewidth=0.1)
    ax.add_patch(polygon)
    
sm = plt.cm.ScalarMappable(cmap=colormap, norm=norm)
cb = plt.colorbar(sm, ax=ax)
cb.set_label("Patch opening (m)", fontsize=15)

plt.title('Patch opening')



#------------------------------------------------------------------------------------------------


ax = fig.add_subplot(1, 2, 2, projection=ccrs.PlateCarree())
 
c = plt.contourf(lon, lat, Z, colornum, cmap=cm,\
               zorder=1, transform = ccrs.PlateCarree())

d = plt.contour(bathy_x, bathy_y, bathy_z, levels, transform = ccrs.PlateCarree(),colors='black', \
               linewidths = 1, linestyles='solid', zorder=2, alpha = .6);


ax = plt.gca()
ax.set_xlim(xlim)
ax.set_ylim(ylim);

ax.set_yticks(np.linspace(ylim[0]+0.02, ylim[1]-0.01, 4), crs = ccrs.PlateCarree())
lat_formatter = LatitudeFormatter()
ax.yaxis.set_major_formatter(lat_formatter)
ax.set_xticks(np.linspace(xlim[0]+0.02, xlim[1]-0.015,4), crs = ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
ax.xaxis.set_major_formatter(lon_formatter)
plt.yticks(rotation=90)

mpr_x, mpr_y = zip(*observations[:len(mpr)])
mpr_lon, mpr_lat = utm_to_latlon_vec(mpr_x, mpr_y, zone_number, zone_letter)

mpr_z_data = np.array(observations_df["z_displacement"][:len(mpr)])
mpr_z_synth = d_syn[:len(mpr)]

for xi, yi, zi in zip(mpr_lon, mpr_lat, mpr_z_data):
    q = plt.quiver(xi, yi, 0, zi, transform=ccrs.PlateCarree(), color='red', scale=7, 
                   headlength=4, headaxislength=3, width=0.015, 
                   zorder=4, edgecolor='white', linewidth=0.9)

for xi, yi, zi in zip(mpr_lon, mpr_lat, mpr_z_synth):
    r = plt.quiver(xi, yi, 0, zi, transform=ccrs.PlateCarree(), color='blue', scale=7, 
                   headlength=7, headaxislength=6, width=0.009, 
                   zorder=4, edgecolor='white', linewidth=0.9)

plt.plot(mmr_x, mmr_y,'--', transform = ccrs.PlateCarree(), color='yellow', alpha=1)
   
ax.add_patch(mpatches.Rectangle(xy=[0.68, 0.68],width=0.3,height=0.3,facecolor='white', transform=ax.transAxes, \
                    zorder=3, edgecolor='black', alpha=0.6))
qk = plt.quiverkey(q, -129.9475, 46.012, 1, 'Data', angle = 90, coordinates='data', labelpos = 'N', \
                  labelsep=0.55, zorder=4, edgecolor='white', linewidth=1);
qk2 = plt.quiverkey(r, -129.926, 46.012, 1, 'Model', angle = 90, coordinates='data', labelpos = 'N', \
                  labelsep=0.55, zorder=4, edgecolor='white', linewidth=1);
plt.text(-129.946, 45.993, '1 meter', zorder=4, fontsize = 15);

cb = plt.colorbar(c);
cb.set_label('Modeled vertical displacement (m)', fontsize=15)
ax.add_patch(mpatches.Rectangle(xy=[0.02, 0.02],width=0.48,height=0.12,facecolor='white', transform=ax.transAxes, \
                    zorder=3, edgecolor='black', alpha=0.6))
plt.text(-130.06, 45.885, 'MPR rmse: '+str("{:.3f}".format(rmse_mpr))+' m', color='black', fontsize=18);
plt.text(-130.06, 45.877, 'AUV rmse: '+str("{:.3f}".format(rmse_auv))+' m', color='black', fontsize=18);

plt.title('Modeled displacement')

# Read out the model data to file

In [ ]:
with h5py.File('model3b_joint_2016_2020.h5', 'w') as hf:
    # Save rectangles
    hf.create_dataset("rectangles", data=rectangles, dtype='float64')
    
    # Save m
    hf.create_dataset("m", data=m, dtype='float64')
    
    # Save lat, lon, Z
    hf.create_dataset("lat", data=lat, dtype='float64')
    hf.create_dataset("lon", data=lon, dtype='float64')
    hf.create_dataset("Z", data=Z, dtype='float64')
    hf.create_dataset("UX",         data=UX,         dtype='float64')
    hf.create_dataset("UY",         data=UY,         dtype='float64')
    
    # Save synthetic mpr data
    hf.create_dataset("mpr_lon", data=mpr_lon, dtype='float64')
    hf.create_dataset("mpr_lat", data=mpr_lat, dtype='float64')
    hf.create_dataset("mpr_z", data=mpr_z_synth, dtype='float64')
    
    # Save rmse_auv, rmse_mpr, and dv
    hf.attrs['rmse_auv'] = rmse_auv
    hf.attrs['rmse_mpr'] = rmse_mpr
    hf.attrs['dv'] = dv_tot